In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chisquare
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier


import imblearn
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_selection import chi2
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier
import os


## Exploratory Data Analysis

In [2]:
#Reading the data from kaggle local storage
df = pd.read_csv("/kaggle/input/diabetes-health-indicators-dataset/diabetes_binary_health_indicators_BRFSS2015.csv")
print(df.columns)
print(df.info())

In [3]:
# Checking for the presence of nulls
df.isna().sum()

In [4]:
# Predict Columns and the Target columns bifurcation
target = 'Diabetes_binary'
feats = [col for col in df.columns if col!=target]

X = df[feats]
y=df[target]

In [5]:
# Number of unique values in each column! All the columns expect BMI are discrete whereas BMI is an approximation
col_types = {}
for col in df.columns:
    if(col != target):
        col_types[col] = df.groupby([col]).count()[target].shape[0]
print(col_types)

In [6]:
fig,ax=plt.subplots(11,2,figsize = (30,50))
i=0
row = 0
col = 0
for col in df.columns:
    if(col!=target):
        
        r = int(i/2)
        c = i % 2
        if(col_types[col]>2):
            sns.scatterplot(df[target],df[col],ax=ax[r][c])
        if(col_types[col]<=2):
            sns.barplot(df[target],df[col],ax=ax[r][c])
        i=i+1
            
plt.show()           
print('The disparity in the heights of bars indicate the dependence between of the target and predictor variables.Higher the disparity higher the importance of the feature') 

## Feature Dependence
Since most of the columns are categorical it is not advisable to use corelation to eliminate any features and hence we are resorting to Chi-Square test ; Mutual Information Score

In [7]:
# HighCol, Stroke,HeartDiseaseorAttack seem to be dependent intuitively
# Performing Chi-Square test
def chi2_independence(col1,col2):
    vals = X.groupby([col1,col2]).count()['BMI'].values
    X.groupby([col1,col2]).count()['BMI']
    col1_p0 = X[X[col1]==0].shape[0]/X.shape[0]
    col2_p0 = X[X[col2]==0].shape[0]/X.shape[0]
    exp =  [X.shape[0] *col1_p0*col2_p0,X.shape[0] *col1_p0*(1-col2_p0),X.shape[0] *(1-col1_p0)*col2_p0,X.shape[0] *(1-col1_p0)*(1-col2_p0)]
    return stats.chisquare(vals,exp).pvalue

p1 = chi2_independence('Stroke','HeartDiseaseorAttack')
print(p1)
p1 = chi2_independence('Stroke','HighChol')
print(p1)
p1 = chi2_independence('HighBP','HighChol')
print(p1)

In [8]:
# Mutual Information Score
print(normalized_mutual_info_score(X['Stroke'],X['HeartDiseaseorAttack'] ))
print(normalized_mutual_info_score(X['Stroke'],X['HighChol'] ))
print(normalized_mutual_info_score(X['HighBP'],X['HighChol'] ))
print(normalized_mutual_info_score(X['PhysActivity'],X['HighChol'] ))
print(normalized_mutual_info_score(X['PhysActivity'],X['HighChol'] ))

In [9]:
print(X.corr().unstack().sort_values().drop_duplicates().head(5))
print(X.corr().unstack().sort_values(ascending=False).drop_duplicates().head(5))
print('Columns are not heavily linearly correlated. So cant use this to filter out any!')

## Hypothesis Testing




#### **Q1** - CDC has a simple [form](https://www.cdc.gov/prediabetes/takethetest/) to help a candidate understand whether he/she is diabetic. If the candidate is "safe" with respect to the select features the result is that the candidate is not diabetic. We want to validate this hypothesis using the data we have
#### **Assumptions**: Obviously this wouldnt be 100% accurate but with an alpha of 1-2% would this be accurate?

#### **Steps**
1. Choose the hypothesis test
2. Data Preparation
3. Perform the test
4. Conclude
5. Explanation

In [10]:
# Preparing the data for hypothesis testing - Making them all booleans
ht_columns = ['Age','GenHlth','PhysActivity','BMI','Smoker']
ht = X[ht_columns]

ht['Age'] = (ht['Age']>4).astype('int')
ht['GenHlth'] = (ht['GenHlth']<3).astype('int')
ht['BMI'] = ((ht['BMI']<=25) & (ht['BMI']>=18)).astype('int')
ht['Sum']=ht.sum(axis=1)
ht['target']= y
ht.head()

In [11]:
#"Sum" column being 0 represents that we are in the "safe-zone" with respect to all the features picked by CDC
ht['safe_feat'] = (ht['Sum']<1).astype('int')
freq_obs = [ht[(ht['safe_feat']==1) & (ht['target']==1)].shape[0],ht[(ht['safe_feat']==1) & (ht['target']==0)].shape[0]]
#Because the expected is that "safe-zone" is free of diabetes
#Instead of using "0" in the expected as formulas might have issues we take it as 10
freq_exp = [10,sum(freq_obs)-10]

chisquare(freq_obs).pvalue

In [12]:
print(freq_obs)

##### Conclusion
pValue is quite low. So we can reject the null hypothesis and conclude that there is a significant impact 

### Q2 - Does it make sense to have different models for younger individuals vs older individuals?
1. Splitting around the age of 40 years let us check this.
2. Build different models for the datasets
3. Compare feature importances in the resultant models using paired t-test
(If the feature importances differ for younger and older individuals we need different models)

In [13]:

clf_young = DecisionTreeClassifier(random_state=0)
clf_old = DecisionTreeClassifier(random_state=0)
col_name = 'Age'
cur_feats = [col for col in feats if col!=col_name]
#Data split to build two different models
X_young = X[X[col_name]<=4][cur_feats]
y_young = y[X[col_name]<=4]
X_old = X[X[col_name]>4][cur_feats]
y_old = y[X[col_name]>4]
#Data Frame with feature importance results
model_diff=pd.DataFrame()

fig,ax=plt.subplots(1,2,figsize=(30,10))
#Model for younger
clf_young.fit(X_young,y_young)
feat_imp_young=pd.Series(clf_young.feature_importances_)
model_diff['young'] = feat_imp_young
feat_imp_young.index=cur_feats
feat_imp_young = feat_imp_young.sort_values(ascending=False)
#Plotting feature importances
sns.barplot(y=feat_imp_young.index,x=feat_imp_young,ax=ax[0])

#Model for older
clf_old.fit(X_old,y_old)
feat_imp_old=pd.Series(clf_old.feature_importances_)
model_diff['old'] = feat_imp_old
feat_imp_old.index=cur_feats
feat_imp_old = feat_imp_old.sort_values(ascending=False)
#plotting feat importances
sns.barplot(y=feat_imp_old.index,x=feat_imp_old,ax=ax[1])

model_diff.index=cur_feats

In [14]:
stats.ttest_rel(model_diff['young'], model_diff['old'])

##### Conclusion:
p Value is very high and hence we know that the models are not very different and hence we do not need to split and build different for younger and older individuals

### Q3 - Does it make sense to have different models for female individuals vs male individuals?
1. Splitting the data for male and female.
2. Build different models for the datasets
3. Compare feature importances in the resultant models using paired t-test
(If the feature importances differ for female and male individuals we need different models)

In [15]:
clf_male = DecisionTreeClassifier(random_state=0)
clf_female = DecisionTreeClassifier(random_state=0)

#Data split to build two different models
col_name = 'Sex'
cur_feats = [col for col in feats if col!=col_name]
X_female = X[X[col_name]==0][cur_feats]
y_female = y[X[col_name]==0]
X_male = X[X[col_name]==1][cur_feats]
y_male = y[X[col_name]==1]
#Data Frame with feature importance results
model_diff=pd.DataFrame()

fig,ax=plt.subplots(1,2,figsize=(30,10))
#Model for female
clf_female.fit(X_female,y_female)
feat_imp_female=pd.Series(clf_female.feature_importances_)
model_diff['female'] = feat_imp_female
feat_imp_female.index=cur_feats
feat_imp_female = feat_imp_female.sort_values(ascending=False)
#Plotting feature importances
sns.barplot(y=feat_imp_female.index,x=feat_imp_female,ax=ax[0])

#Model for male
clf_male.fit(X_male,y_male)
feat_imp_male=pd.Series(clf_male.feature_importances_)
model_diff['male'] = feat_imp_male
feat_imp_male.index=cur_feats
feat_imp_male = feat_imp_male.sort_values(ascending=False)
#plotting feat importances
sns.barplot(y=feat_imp_male.index,x=feat_imp_male,ax=ax[1])

model_diff.index=cur_feats

In [16]:
stats.ttest_rel(model_diff['female'], model_diff['male'])

### Q4 - Does it make sense to have different models for Smoker individuals vs Non-Smoker individuals?
1. Splitting the data for male and female.
2. Build different models for the datasets
3. Compare feature importances in the resultant models using paired t-test
(If the feature importances differ for female and male individuals we need different models)

In [17]:


clf_male = DecisionTreeClassifier(random_state=0)
clf_female = DecisionTreeClassifier(random_state=0)

#Data split to build two different models
col_name = 'Smoker'
cur_feats = [col for col in feats if col!=col_name]
X_female = X[X[col_name]==0][cur_feats]
y_female = y[X[col_name]==0]
X_male = X[X[col_name]==1][cur_feats]
y_male = y[X[col_name]==1]
#Data Frame with feature importance results
model_diff=pd.DataFrame()

fig,ax=plt.subplots(1,2,figsize=(30,10))
#Model for female
clf_female.fit(X_female,y_female)
feat_imp_female=pd.Series(clf_female.feature_importances_)
model_diff['non_smoker'] = feat_imp_female
feat_imp_female.index=cur_feats
feat_imp_female = feat_imp_female.sort_values(ascending=False)
#Plotting feature importances
sns.barplot(y=feat_imp_female.index,x=feat_imp_female,ax=ax[0])

#Model for male
clf_male.fit(X_male,y_male)
feat_imp_male=pd.Series(clf_male.feature_importances_)
model_diff['smoker'] = feat_imp_male
feat_imp_male.index=cur_feats
feat_imp_male = feat_imp_male.sort_values(ascending=False)
#plotting feat importances
sns.barplot(y=feat_imp_male.index,x=feat_imp_male,ax=ax[1])

model_diff.index=cur_feats

stats.ttest_rel(model_diff['non_smoker'], model_diff['smoker'])

In [18]:
X.groupby(['NoDocbcCost']).count()

### Q5 - Does it make sense to have different models for NoDoctorBcos Of Cost individuals vs No Cost Problem individuals?
1. Splitting the data for male and female.
2. Build different models for the datasets
3. Compare feature importances in the resultant models using paired t-test
(If the feature importances differ for female and male individuals we need different models)

In [19]:


clf_male = DecisionTreeClassifier(random_state=0)
clf_female = DecisionTreeClassifier(random_state=0)

#Data split to build two different models
col_name = 'NoDocbcCost'
cur_feats = [col for col in feats if col!=col_name]
X_female = X[X[col_name]==0][cur_feats]
y_female = y[X[col_name]==0]
X_male = X[X[col_name]==1][cur_feats]
y_male = y[X[col_name]==1]
#Data Frame with feature importance results
model_diff=pd.DataFrame()

fig,ax=plt.subplots(1,2,figsize=(30,10))
#Model for female
clf_female.fit(X_female,y_female)
feat_imp_female=pd.Series(clf_female.feature_importances_)
model_diff['costProb'] = feat_imp_female
feat_imp_female.index=cur_feats
feat_imp_female = feat_imp_female.sort_values(ascending=False)
#Plotting feature importances
sns.barplot(y=feat_imp_female.index,x=feat_imp_female,ax=ax[0])

#Model for male
clf_male.fit(X_male,y_male)
feat_imp_male=pd.Series(clf_male.feature_importances_)
model_diff['noCostProb'] = feat_imp_male
feat_imp_male.index=cur_feats
feat_imp_male = feat_imp_male.sort_values(ascending=False)
#plotting feat importances
sns.barplot(y=feat_imp_male.index,x=feat_imp_male,ax=ax[1])

model_diff.index=cur_feats

stats.ttest_rel(model_diff['costProb'], model_diff['noCostProb'])

## Data Preparation
Using Random Forest
1. Perform classification with PCA
2. Perform calssification without PCA

In [20]:
#Split into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# Split train into train and CV
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.33, random_state=42)
#Oversample data
oversample = RandomOverSampler(sampling_strategy=0.5)
X_over, y_over = oversample.fit_resample(X_train, y_train)

In [21]:
#Scaling Data - Standard Scalar before PCA
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

#Performing PCA where we just pick the features that cover 85% of the variance
pca = PCA(n_components=0.85)
pca.fit(X_train_std)
len(pca.components_)
X_train_std_pca = pca.transform(X_train_std)
X_test_std_pca = pca.transform(X_test_std)

X_over_pca, y_over_pca = oversample.fit_resample(X_train_std_pca, y_train)

## Feature Importance

In [22]:
feat_imp=np.zeros(len(feats))

cnt=0
for n in range(50,200,50):
#     for d in range(5,20,5):
            for m in range(40,60,10):
                    for m2 in range(10,30,10):
                        clf = RandomForestClassifier(n_estimators = n,min_samples_split=m,min_samples_leaf=m2)
                        clf.fit(X_over,y_over)
                        print(f'Esimators : {n}, min_samples_split : {m},min_samples_leaf: {m2}')
                        print(f'CV Score - {f1_score(clf.predict(X_cv),y_cv,average="weighted")}')
                        print(f'Train Score - {f1_score(clf.predict(X_train),y_train,average="weighted")}')
                        feat_imp = feat_imp+clf.feature_importances_
                        cnt=cnt+1
feat_imp = feat_imp/cnt
feat_imp=pd.Series(feat_imp)

#plotting feat importances
feat_imp.index=feats
feat_imp = feat_imp.sort_values(ascending=False)
sns.barplot(y=feat_imp.index,x=feat_imp)

## Possible Feature Elmination

In [23]:
A = X.copy()
A['Food'] = A['Fruits']+A['Veggies']
A.drop(columns=['Fruits','Veggies'],inplace=True)
A['lifeStyle'] = A['Smoker']+A['HvyAlcoholConsump']
A.drop(columns=['Smoker','HvyAlcoholConsump'],inplace=True)
A.drop(columns=['CholCheck','NoDocbcCost','AnyHealthcare'],inplace=True)

A_train,A_test,Ay_train,Ay_test = train_test_split(A,y,test_size=0.33,random_state=0)
A_over, Ay_over = oversample.fit_resample(A_train, Ay_train)

## Modeling

### Baseline Model

In [24]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)

print(f'Test Score - {f1_score(dummy_clf.predict(X_test),y_test)}')
print(f'Train Score - {f1_score(dummy_clf.predict(X_train),y_train)}')

print(f'Test Score - {accuracy_score(dummy_clf.predict(X_test),y_test)}')
print(f'Train Score - {accuracy_score(dummy_clf.predict(X_train),y_train)}')

print(f'Test Score - {precision_score(dummy_clf.predict(X_test),y_test)}')
print(f'Train Score - {precision_score(dummy_clf.predict(X_train),y_train)}')
print(f'Test Score - {recall_score(dummy_clf.predict(X_test),y_test)}')
print(f'Train Score - {recall_score(dummy_clf.predict(X_train),y_train)}')

## Impact of OverSampling the data for imbalanced classification problems
## Impact of PCA on the data like this which is mostly categorical

In [25]:
print('Logistic Regression Classification')
print('--------------------------------------------------------------')
#Decision Tree as a model slightly better than baseline
print('Imbalanced')
clf = LogisticRegression(solver='liblinear')
clf.fit(X_train,y_train)
print(f'Test Score - {f1_score(clf.predict(X_test),y_test)}')
print(f'Train Score - {f1_score(clf.predict(X_train),y_train)}')

# Comparing Decision Tree performance on imbalanced data vs Oversampled data
print('Oversampled')
clf = LogisticRegression(solver='liblinear')
clf.fit(X_over,y_over)
print(f'Test Score - {f1_score(clf.predict(X_test),y_test)}')
print(f'Train Score - {f1_score(clf.predict(X_train),y_train)}')

print('PCA and oversampled')
clf = LogisticRegression(solver='liblinear')
clf.fit(X_over_pca,y_over_pca)
print(f'Test Score - {f1_score(clf.predict(X_test_std_pca),y_test)}')
print(f'Train Score - {f1_score(clf.predict(X_train_std_pca),y_train)}')

print('Feature Selection')
clf.fit(A_over,Ay_over)
print(f'Test Score - {f1_score(clf.predict(A_test),Ay_test)}')
print(f'Train Score - {f1_score(clf.predict(A_over),Ay_over)}')

In [26]:
print('DecisionTree Classification')
print('--------------------------------------------------------------')

#Decision Tree as a model slightly better than baseline
print('Imbalanced')
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
print(f'Test Score - {f1_score(clf.predict(X_test),y_test)}')
print(f'Train Score - {f1_score(clf.predict(X_train),y_train)}')

#Comparing Decision Tree performance on imbalanced data vs Oversampled data
print('Oversampled')
clf = DecisionTreeClassifier()
clf.fit(X_over,y_over)
print(f'Test Score - {f1_score(clf.predict(X_test),y_test)}')
print(f'Train Score - {f1_score(clf.predict(X_train),y_train)}')

print('PCA and oversampled')
clf.fit(X_over_pca,y_over_pca)
print(f'Test Score - {f1_score(clf.predict(X_test_std_pca),y_test)}')
print(f'Train Score - {f1_score(clf.predict(X_train_std_pca),y_train)}')

print('Feature Selection')
clf.fit(A_over,Ay_over)
print(f'Test Score - {f1_score(clf.predict(A_test),Ay_test)}')
print(f'Train Score - {f1_score(clf.predict(A_over),Ay_over)}')

In [27]:
print('RandomForestClassification')
print('--------------------------------------------------------------')

#Is Random Forest in-general performing better
print('Imbalanced')
clf = RandomForestClassifier()
clf.fit(X_train,y_train)
print(f'Test Score - {f1_score(clf.predict(X_test),y_test)}')
print(f'Train Score - {f1_score(clf.predict(X_train),y_train)}')

# Is Random Forest in-general performing better
print('Oversampled')
clf = RandomForestClassifier()
clf.fit(X_over,y_over)
print(f'Test Score - {f1_score(clf.predict(X_test),y_test)}')
print(f'Train Score - {f1_score(clf.predict(X_train),y_train)}')

print('PCA and oversampled')
clf.fit(X_over_pca,y_over_pca)
print(f'Test Score - {f1_score(clf.predict(X_test_std_pca),y_test)}')
print(f'Train Score - {f1_score(clf.predict(X_train_std_pca),y_train)}')

print('Feature Selection')
clf.fit(A_over,Ay_over)
print(f'Test Score - {f1_score(clf.predict(A_test),Ay_test)}')
print(f'Train Score - {f1_score(clf.predict(A_over),Ay_over)}')

In [28]:
print('XGBoost Classifier')
print('--------------------------------------------------------------')

#Is Random Forest in-general performing better
print('Imbalanced')
model = XGBClassifier(eval_metric='auc',scale_pos_weight = (y_over.shape[0] / y.sum())-1)
model.fit(X_over, y_over)
print(f'Test Score - {f1_score(model.predict(X_test),y_test)}')
print(f'Train Score - {f1_score(model.predict(X_train),y_train)}')

print('Oversampled')
model = XGBClassifier(eval_metric='auc',scale_pos_weight = (y_over.shape[0] / y.sum())-1)
model.fit(X_train, y_train)
print(f'Test Score - {f1_score(model.predict(X_test),y_test)}')
print(f'Train Score - {f1_score(model.predict(X_train),y_train)}')

print('PCA and oversampled')
model.fit(X_over_pca,y_over_pca)
print(f'Test Score - {f1_score(clf.predict(X_test_std_pca),y_test)}')
print(f'Train Score - {f1_score(clf.predict(X_train_std_pca),y_train)}')

print('Feature Selection')
model.fit(A_over,Ay_over)
print(f'Test Score - {f1_score(model.predict(A_test),Ay_test)}')
print(f'Train Score - {f1_score(model.predict(A_over),Ay_over)}')

### Hyper Parameter Tuning using Cross Validation data with Random Forest

In [29]:
# Cross Validation of Random Forest
for n in range(50,200,50):
            for m in range(20,60,10):
                    for m2 in range(10,50,10):
                        clf = RandomForestClassifier(n_estimators = n,min_samples_split=m,min_samples_leaf=m2)
                        clf.fit(X_over,y_over)
                        print(f'Esimators : {n}, min_samples_split : {m},min_samples_leaf: {m2}')
                        print(f'CV Score - {f1_score(clf.predict(X_cv),y_cv,average="weighted")}')
                        print(f'Train Score - {f1_score(clf.predict(X_train),y_train,average="weighted")}')

### Hyper Parameter Tuning using Cross Validation data with XGBoost

In [30]:
# XGBoost Hyper Parameter tuning
for ss in range(5,9):
        for cs in range(5,9):

            model = XGBClassifier(learning_rate =0.1,n_estimators=50,gamma=0,colsample_bytree=(cs/10),subsample=(ss/10),objective= 'binary:logistic',seed=27)
            model.fit(X_over,y_over)
            print(f'{ss},{cs}')
            print(f'Test Score - {f1_score(model.predict(X_test),y_test,average="weighted")}')
            print(f'Train Score - {f1_score(model.predict(X_over),y_over,average="weighted")}')

### Hyper Parameter Tuning using Cross Validation data with MLP

In [31]:
# Neural Network Hyper Parameter Tuning
for h1 in np.arange(1, 6):
     for h2 in np.arange(1, 5):
        clf = MLPClassifier(solver='sgd',alpha=1e-05,hidden_layer_sizes=(h1,h2), random_state=1,max_iter=100)
        clf.fit(X_train_std, y_train)
        print(f'{h1},{h2}')
        print(f1_score(clf.predict(X_test_std),y_test,average='weighted'))
        print(f1_score(clf.predict(X_train_std),y_train,average='weighted'))
                

## Hyper Parameter Tuning with 80% of the Features

In [32]:
# for n in range(50,200,50):
#             for m in range(20,60,10):
#                     for m2 in range(10,50,10):
#                         clf = RandomForestClassifier(n_estimators = n,min_samples_split=m,min_samples_leaf=m2)
#                         clf.fit(A_over,Ay_over)
#                         print(f'Esimators : {n}, min_samples_split : {m},min_samples_leaf: {m2}')
#                         print(f'CV Score - {f1_score(clf.predict(A_test),Ay_test,average="weighted")}')
#                         print(f'Train Score - {f1_score(clf.predict(A_over),Ay_over,average="weighted")}')
                        

In [33]:
# #XGBoost Hyper Parameter tuning
# for ss in range(5,9):
#         for cs in range(5,9):

#             model = XGBClassifier(learning_rate =0.1,n_estimators=50,gamma=0,colsample_bytree=(cs/10),subsample=(ss/10),objective= 'binary:logistic',seed=27)
#             model.fit(A_over,Ay_over)
#             print(f'{ss},{cs}')
#             print(f'Test Score - {f1_score(model.predict(A_test),Ay_test,average="weighted")}')
#             print(f'Train Score - {f1_score(model.predict(A_over),Ay_over,average="weighted")}')
        